<a href="https://colab.research.google.com/github/Hu-Hao/quant-learning/blob/main/examples/colab_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quantitative Trading Framework - Colab Example

This notebook demonstrates how to use the quantitative trading framework to run backtests with real market data from Yahoo Finance.

## Features Demonstrated:
- Installing the library directly from GitHub
- Fetching real Apple (AAPL) stock data from Yahoo Finance
- Running backtests with multiple trading strategies
- Analyzing performance with comprehensive metrics
- Visualizing results with charts and plots

## 1. Installation and Setup

First, let's install the required dependencies and clone our trading framework from GitHub.

In [ ]:
# Install required packages
!pip install yfinance pandas numpy matplotlib seaborn

# Clone the quantitative trading framework from GitHub
!git clone https://github.com/Hu-Hao/quant-learning.git

# Add the project to Python path
import sys
sys.path.append('/content/quant-learning')

print("✅ Installation completed successfully!")

## 2. Import Libraries

Import all necessary modules from our trading framework and other required libraries.

In [ ]:
# Standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Import our quantitative trading framework
from quant_trading.strategies.moving_average import MovingAverageStrategy
from quant_trading.strategies.momentum import MomentumStrategy
from quant_trading.strategies.mean_reversion import MeanReversionStrategy
from quant_trading.backtesting.engine import BacktestEngine
from quant_trading.utils.visualization import PerformanceVisualizer

# Set up plotting
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)

print("✅ All libraries imported successfully!")

## 3. Data Fetching

Let's fetch real Apple (AAPL) stock data from Yahoo Finance for the past 2 years.

In [ ]:
# Define the stock symbol and date range
symbol = "AAPL"
end_date = datetime.now()
start_date = end_date - timedelta(days=2*365)  # 2 years of data

print(f"Fetching {symbol} data from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}...")

# Fetch data from Yahoo Finance
ticker = yf.Ticker(symbol)
data = ticker.history(start=start_date, end=end_date)

# Convert column names to lowercase for compatibility with our framework
data.columns = [col.lower() for col in data.columns]

# Display basic information about the data
print(f"\n📊 Data Summary:")
print(f"   • Total trading days: {len(data)}")
print(f"   • Date range: {data.index[0].strftime('%Y-%m-%d')} to {data.index[-1].strftime('%Y-%m-%d')}")
print(f"   • Price range: ${data['low'].min():.2f} - ${data['high'].max():.2f}")
print(f"   • Current price: ${data['close'].iloc[-1]:.2f}")

# Display first few rows
print(f"\n📈 First 5 rows:")
print(data.head())

## 4. Data Visualization

Let's visualize the Apple stock price data before running our backtests.

In [ ]:
# Create a comprehensive price chart
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# 1. Price chart with moving averages
ax1.plot(data.index, data['close'], label='Close Price', linewidth=1, color='blue')
ax1.plot(data.index, data['close'].rolling(20).mean(), label='20-day MA', linewidth=1, color='orange')
ax1.plot(data.index, data['close'].rolling(50).mean(), label='50-day MA', linewidth=1, color='red')
ax1.set_title(f'{symbol} Stock Price with Moving Averages')
ax1.set_ylabel('Price ($)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Volume chart
ax2.bar(data.index, data['volume'], alpha=0.7, color='green')
ax2.set_title(f'{symbol} Trading Volume')
ax2.set_ylabel('Volume')
ax2.grid(True, alpha=0.3)

# 3. Daily returns
returns = data['close'].pct_change().dropna()
ax3.plot(data.index[1:], returns * 100, linewidth=0.8, color='purple')
ax3.axhline(y=0, color='black', linestyle='-', alpha=0.3)
ax3.set_title(f'{symbol} Daily Returns')
ax3.set_ylabel('Return (%)')
ax3.grid(True, alpha=0.3)

# 4. Returns distribution
ax4.hist(returns * 100, bins=50, alpha=0.7, color='teal', edgecolor='black')
ax4.axvline(returns.mean() * 100, color='red', linestyle='--', label=f'Mean: {returns.mean()*100:.2f}%')
ax4.set_title(f'{symbol} Returns Distribution')
ax4.set_xlabel('Daily Return (%)')
ax4.set_ylabel('Frequency')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print some basic statistics
print(f"\n📊 Basic Statistics:")
print(f"   • Average daily return: {returns.mean()*100:.3f}%")
print(f"   • Daily volatility: {returns.std()*100:.3f}%")
print(f"   • Annualized volatility: {returns.std()*np.sqrt(252)*100:.1f}%")
print(f"   • Total return: {(data['close'].iloc[-1]/data['close'].iloc[0] - 1)*100:.1f}%")

## 5. Strategy Setup

Now let's set up our three different trading strategies with different parameters.

In [ ]:
# Define our trading strategies
strategies = {
    'Moving Average (Fast)': MovingAverageStrategy(
        short_window=10,
        long_window=30,
        quantity=100
    ),
    'Moving Average (Slow)': MovingAverageStrategy(
        short_window=20,
        long_window=50,
        quantity=100
    ),
    'Momentum': MomentumStrategy(
        lookback_period=20,
        momentum_threshold=0.02,  # 2% threshold
        quantity=100,
        volatility_filter=True
    ),
    'Mean Reversion': MeanReversionStrategy(
        window=20,
        entry_threshold=2.0,  # 2 standard deviations
        exit_threshold=0.5,
        quantity=100,
        use_bollinger_bands=True
    )
}

# Display strategy information
print("🎯 Trading Strategies Setup:")
for name, strategy in strategies.items():
    params = strategy.get_parameters()
    print(f"\n   • {name}:")
    for key, value in params.items():
        print(f"     - {key}: {value}")

print("\n✅ All strategies configured successfully!")

## 6. Run Backtests

Let's run backtests for each strategy and compare their performance.

In [ ]:
# Set up backtesting parameters
initial_capital = 100000  # $100,000
commission = 0.001        # 0.1% commission
slippage = 0.001         # 0.1% slippage

# Store results for comparison
backtest_results = {}

print(f"🚀 Running backtests with ${initial_capital:,} initial capital...\n")

for strategy_name, strategy in strategies.items():
    print(f"📈 Testing {strategy_name}...")
    
    # Create a new backtest engine for each strategy
    engine = BacktestEngine(
        initial_capital=initial_capital,
        commission=commission,
        slippage=slippage,
        max_position_size=0.95,  # Allow up to 95% of capital per position
        risk_free_rate=0.02
    )
    
    # Run the backtest
    engine.run_backtest(data, strategy)
    
    # Calculate performance metrics
    performance = engine.get_performance_summary()
    
    # Store results
    backtest_results[strategy_name] = {
        'engine': engine,
        'performance': performance,
        'portfolio_values': engine.portfolio_values.copy(),
        'trades': engine.trades.copy()
    }
    
    # Print quick summary
    final_value = engine.portfolio_values[-1] if engine.portfolio_values else initial_capital
    total_return = (final_value / initial_capital - 1) * 100
    num_trades = len(engine.trades)
    
    print(f"   ✅ Completed! Final value: ${final_value:,.2f} ({total_return:+.1f}%), Trades: {num_trades}")

print(f"\n🎉 All backtests completed successfully!")

## 7. Performance Analysis

Let's analyze and compare the performance of all strategies.

In [ ]:
# Create performance comparison table
performance_df = pd.DataFrame()

for strategy_name, results in backtest_results.items():
    perf = results['performance']
    
    # Calculate additional metrics
    final_value = results['portfolio_values'][-1] if results['portfolio_values'] else initial_capital
    total_return = (final_value / initial_capital - 1) * 100
    
    performance_df[strategy_name] = {
        'Final Value ($)': f"{final_value:,.2f}",
        'Total Return (%)': f"{total_return:.2f}",
        'Annualized Return (%)': f"{perf.get('annualized_return', 0)*100:.2f}",
        'Volatility (%)': f"{perf.get('volatility', 0)*100:.2f}",
        'Sharpe Ratio': f"{perf.get('sharpe_ratio', 0):.3f}",
        'Max Drawdown (%)': f"{perf.get('max_drawdown', 0)*100:.2f}",
        'Number of Trades': len(results['trades']),
        'Win Rate (%)': f"{perf.get('win_rate', 0)*100:.1f}"
    }

# Display performance table
print("📊 PERFORMANCE COMPARISON SUMMARY")
print("=" * 60)
print(performance_df.T.to_string())

# Calculate buy and hold benchmark
buy_hold_return = (data['close'].iloc[-1] / data['close'].iloc[0] - 1) * 100
buy_hold_final = initial_capital * (1 + buy_hold_return/100)

print(f"\n📈 BENCHMARK COMPARISON")
print("=" * 40)
print(f"Buy & Hold {symbol}: {buy_hold_return:.2f}% (${buy_hold_final:,.2f})")

# Find best performing strategy
best_strategy = max(backtest_results.keys(), 
                   key=lambda x: backtest_results[x]['portfolio_values'][-1] if backtest_results[x]['portfolio_values'] else 0)
best_return = (backtest_results[best_strategy]['portfolio_values'][-1] / initial_capital - 1) * 100

print(f"\n🏆 BEST STRATEGY: {best_strategy}")
print(f"   Return: {best_return:.2f}% vs Buy & Hold: {buy_hold_return:.2f}%")
print(f"   Outperformance: {best_return - buy_hold_return:+.2f}%")

## 8. Performance Visualization

Let's create comprehensive visualizations to compare strategy performance.

In [ ]:
# Create comprehensive performance visualization
fig = plt.figure(figsize=(16, 12))

# 1. Portfolio value comparison
ax1 = plt.subplot(2, 3, 1)
for strategy_name, results in backtest_results.items():
    if results['portfolio_values']:
        dates = data.index[:len(results['portfolio_values'])]
        ax1.plot(dates, results['portfolio_values'], label=strategy_name, linewidth=2)

# Add buy and hold benchmark
buy_hold_values = initial_capital * (data['close'] / data['close'].iloc[0])
ax1.plot(data.index, buy_hold_values, label='Buy & Hold', linestyle='--', color='black', alpha=0.7)

ax1.set_title('Portfolio Value Over Time')
ax1.set_ylabel('Portfolio Value ($)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Total returns comparison
ax2 = plt.subplot(2, 3, 2)
strategy_returns = []
strategy_names = []

for strategy_name, results in backtest_results.items():
    if results['portfolio_values']:
        total_return = (results['portfolio_values'][-1] / initial_capital - 1) * 100
        strategy_returns.append(total_return)
        strategy_names.append(strategy_name)

# Add buy and hold
strategy_returns.append(buy_hold_return)
strategy_names.append('Buy & Hold')

colors = ['skyblue', 'lightgreen', 'salmon', 'gold', 'lightgray']
bars = ax2.bar(strategy_names, strategy_returns, color=colors[:len(strategy_names)])
ax2.set_title('Total Returns Comparison')
ax2.set_ylabel('Return (%)')
ax2.tick_params(axis='x', rotation=45)

# Add value labels on bars
for bar, value in zip(bars, strategy_returns):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + max(strategy_returns) * 0.01,
             f'{value:.1f}%', ha='center', va='bottom')

ax2.grid(True, alpha=0.3)

# 3. Drawdown analysis
ax3 = plt.subplot(2, 3, 3)
for strategy_name, results in backtest_results.items():
    if results['portfolio_values']:
        values = pd.Series(results['portfolio_values'])
        running_max = values.expanding().max()
        drawdown = (values - running_max) / running_max * 100
        dates = data.index[:len(drawdown)]
        ax3.fill_between(dates, drawdown, 0, alpha=0.3, label=strategy_name)

ax3.set_title('Drawdown Analysis')
ax3.set_ylabel('Drawdown (%)')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Risk-Return scatter plot
ax4 = plt.subplot(2, 3, 4)
returns_list = []
volatility_list = []
names_list = []

for strategy_name, results in backtest_results.items():
    perf = results['performance']
    if perf:
        returns_list.append(perf.get('annualized_return', 0) * 100)
        volatility_list.append(perf.get('volatility', 0) * 100)
        names_list.append(strategy_name)

scatter = ax4.scatter(volatility_list, returns_list, s=100, alpha=0.7, c=range(len(names_list)), cmap='viridis')

for i, name in enumerate(names_list):
    ax4.annotate(name, (volatility_list[i], returns_list[i]), 
                xytext=(5, 5), textcoords='offset points', fontsize=8)

ax4.set_title('Risk vs Return')
ax4.set_xlabel('Volatility (%)')
ax4.set_ylabel('Annualized Return (%)')
ax4.grid(True, alpha=0.3)

# 5. Monthly returns heatmap for best strategy
ax5 = plt.subplot(2, 3, 5)
best_values = pd.Series(backtest_results[best_strategy]['portfolio_values'], 
                       index=data.index[:len(backtest_results[best_strategy]['portfolio_values'])])
monthly_returns = best_values.resample('M').last().pct_change().dropna() * 100

# Create monthly returns table
monthly_table = monthly_returns.groupby([monthly_returns.index.year, monthly_returns.index.month]).first().unstack()
if not monthly_table.empty:
    im = ax5.imshow(monthly_table.values, cmap='RdYlGn', aspect='auto')
    ax5.set_xticks(range(12))
    ax5.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                        'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
    ax5.set_yticks(range(len(monthly_table.index)))
    ax5.set_yticklabels(monthly_table.index)
    ax5.set_title(f'Monthly Returns Heatmap\n({best_strategy})')
    plt.colorbar(im, ax=ax5, label='Return (%)')

# 6. Trade analysis for best strategy
ax6 = plt.subplot(2, 3, 6)
best_trades = backtest_results[best_strategy]['trades']
if best_trades:
    trade_pnls = [trade.pnl for trade in best_trades]
    colors = ['green' if pnl > 0 else 'red' for pnl in trade_pnls]
    ax6.bar(range(len(trade_pnls)), trade_pnls, color=colors, alpha=0.7)
    ax6.axhline(y=0, color='black', linestyle='-', alpha=0.3)
    ax6.set_title(f'Individual Trade P&L\n({best_strategy})')
    ax6.set_xlabel('Trade Number')
    ax6.set_ylabel('P&L ($)')
    ax6.grid(True, alpha=0.3)
else:
    ax6.text(0.5, 0.5, 'No trades executed', ha='center', va='center', transform=ax6.transAxes)
    ax6.set_title(f'Individual Trade P&L\n({best_strategy})')

plt.tight_layout()
plt.show()

## 9. Detailed Trade Analysis

Let's dive deeper into the trade details for our best performing strategy.

In [ ]:
# Analyze trades for the best strategy
best_trades = backtest_results[best_strategy]['trades']

print(f"🔍 DETAILED TRADE ANALYSIS - {best_strategy}")
print("=" * 60)

if best_trades:
    # Convert trades to DataFrame for easier analysis
    trade_data = []
    for i, trade in enumerate(best_trades):
        trade_data.append({
            'Trade #': i + 1,
            'Entry Date': trade.entry_time.strftime('%Y-%m-%d'),
            'Exit Date': trade.exit_time.strftime('%Y-%m-%d'),
            'Side': trade.side,
            'Entry Price': f"${trade.entry_price:.2f}",
            'Exit Price': f"${trade.exit_price:.2f}",
            'Quantity': trade.quantity,
            'P&L': f"${trade.pnl:.2f}",
            'Return %': f"{trade.return_pct*100:.2f}%",
            'Days Held': (trade.exit_time - trade.entry_time).days
        })
    
    trade_df = pd.DataFrame(trade_data)
    print(trade_df.to_string(index=False))
    
    # Calculate trade statistics
    winning_trades = [t for t in best_trades if t.pnl > 0]
    losing_trades = [t for t in best_trades if t.pnl < 0]
    
    total_pnl = sum(t.pnl for t in best_trades)
    avg_pnl = total_pnl / len(best_trades)
    win_rate = len(winning_trades) / len(best_trades) * 100
    
    avg_win = sum(t.pnl for t in winning_trades) / len(winning_trades) if winning_trades else 0
    avg_loss = sum(t.pnl for t in losing_trades) / len(losing_trades) if losing_trades else 0
    profit_factor = abs(sum(t.pnl for t in winning_trades) / sum(t.pnl for t in losing_trades)) if losing_trades else float('inf')
    
    print(f"\n📈 TRADE STATISTICS")
    print(f"   • Total Trades: {len(best_trades)}")
    print(f"   • Winning Trades: {len(winning_trades)} ({win_rate:.1f}%)")
    print(f"   • Losing Trades: {len(losing_trades)}")
    print(f"   • Total P&L: ${total_pnl:.2f}")
    print(f"   • Average P&L per Trade: ${avg_pnl:.2f}")
    print(f"   • Average Win: ${avg_win:.2f}")
    print(f"   • Average Loss: ${avg_loss:.2f}")
    print(f"   • Profit Factor: {profit_factor:.2f}")
    
    # Best and worst trades
    best_trade = max(best_trades, key=lambda t: t.pnl)
    worst_trade = min(best_trades, key=lambda t: t.pnl)
    
    print(f"\n🏆 BEST TRADE: ${best_trade.pnl:.2f} ({best_trade.return_pct*100:.1f}%)")
    print(f"   {best_trade.entry_time.strftime('%Y-%m-%d')} to {best_trade.exit_time.strftime('%Y-%m-%d')}")
    
    print(f"\n💸 WORST TRADE: ${worst_trade.pnl:.2f} ({worst_trade.return_pct*100:.1f}%)")
    print(f"   {worst_trade.entry_time.strftime('%Y-%m-%d')} to {worst_trade.exit_time.strftime('%Y-%m-%d')}")
    
else:
    print("No trades were executed by this strategy.")
    print("This might indicate:")
    print("• The strategy conditions were not met during the backtest period")
    print("• The strategy parameters may need adjustment")
    print("• The market conditions were not suitable for this strategy")

## 10. Conclusions and Next Steps

Let's summarize our findings and suggest next steps for improvement.

In [ ]:
print("🎯 BACKTEST SUMMARY & CONCLUSIONS")
print("=" * 50)

# Calculate key metrics
best_return = (backtest_results[best_strategy]['portfolio_values'][-1] / initial_capital - 1) * 100
benchmark_return = buy_hold_return
outperformance = best_return - benchmark_return

print(f"\n📊 KEY RESULTS:")
print(f"   • Best Strategy: {best_strategy}")
print(f"   • Strategy Return: {best_return:.2f}%")
print(f"   • Buy & Hold Return: {benchmark_return:.2f}%")
print(f"   • Outperformance: {outperformance:+.2f}%")
print(f"   • Annualized Sharpe Ratio: {backtest_results[best_strategy]['performance'].get('sharpe_ratio', 0):.3f}")

# Performance ranking
print(f"\n🏆 STRATEGY RANKING (by Total Return):")
strategy_performance = []
for name, results in backtest_results.items():
    if results['portfolio_values']:
        ret = (results['portfolio_values'][-1] / initial_capital - 1) * 100
        strategy_performance.append((name, ret))

strategy_performance.append(('Buy & Hold (Benchmark)', benchmark_return))
strategy_performance.sort(key=lambda x: x[1], reverse=True)

for i, (name, ret) in enumerate(strategy_performance, 1):
    icon = "🥇" if i == 1 else "🥈" if i == 2 else "🥉" if i == 3 else "  "
    print(f"   {icon} {i}. {name}: {ret:+.2f}%")

print(f"\n💡 KEY INSIGHTS:")
if outperformance > 0:
    print(f"   ✅ Our best strategy outperformed buy-and-hold by {outperformance:.1f}%")
else:
    print(f"   ❌ Buy-and-hold outperformed our best strategy by {abs(outperformance):.1f}%")

# Count profitable strategies
profitable_strategies = sum(1 for _, results in backtest_results.items() 
                          if results['portfolio_values'] and results['portfolio_values'][-1] > initial_capital)

print(f"   • {profitable_strategies}/{len(backtest_results)} strategies were profitable")

# Analyze market conditions
market_trend = "bullish" if benchmark_return > 0 else "bearish"
print(f"   • Market was {market_trend} during the test period ({benchmark_return:+.1f}%)")

print(f"\n🚀 NEXT STEPS FOR IMPROVEMENT:")
print(f"   1. Parameter Optimization:")
print(f"      • Run grid search to find optimal parameters")
print(f"      • Use walk-forward analysis for robust testing")
print(f"   2. Risk Management:")
print(f"      • Add stop-loss and take-profit levels")
print(f"      • Implement position sizing based on volatility")
print(f"   3. Strategy Enhancement:")
print(f"      • Combine multiple strategies (ensemble)")
print(f"      • Add market regime detection")
print(f"   4. Extended Testing:")
print(f"      • Test on multiple assets and time periods")
print(f"      • Include transaction costs and market impact")
print(f"   5. Live Trading Preparation:")
print(f"      • Paper trading validation")
print(f"      • Risk management systems")

print(f"\n🎉 FRAMEWORK FEATURES DEMONSTRATED:")
print(f"   ✅ Real market data integration (Yahoo Finance)")
print(f"   ✅ Multiple strategy implementations")
print(f"   ✅ Realistic execution modeling (slippage, commissions)")
print(f"   ✅ Comprehensive performance analysis")
print(f"   ✅ Professional visualization tools")
print(f"   ✅ Detailed trade tracking and analysis")

print(f"\n📚 To explore more features, check out:")
print(f"   • GitHub Repository: https://github.com/Hu-Hao/quant-learning")
print(f"   • Strategy customization examples")
print(f"   • Advanced risk management techniques")
print(f"   • Multi-asset portfolio backtesting")

## ⚠️ Important Disclaimer

**This notebook is for educational and research purposes only.**

- Past performance does not guarantee future results
- All trading strategies involve risk of loss
- Backtest results may not reflect real-world trading conditions
- Always consult with financial professionals before making investment decisions
- Start with paper trading before risking real capital

**Remember**: The goal is to learn about quantitative trading concepts, not to provide investment advice!